In [1]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_1D import *
from ids import *
%matplotlib inline

In [7]:
from numba import jit
# @jit()
def BF():
    global mat
    fnd = 0
    for i in xrange(1000000):
        # print i
        max_val = 20
        n, m = 10, 6
        mat_int = np.random.uniform( low=-max_val, high=max_val, size=n*m).reshape((n,m)).astype(int)
        mat = mat_int.astype(float)
        try:
            _,_,_,_, info = pluq_ids(mat, debug=False)
        except:
            np.savetxt('except.txt', mat_int)
            print 'except'
            return mat
        if info[1] > 100:
            np.savetxt('contra_{}_{}.txt'.format(fnd, info[1]), mat_int)
            return mat
            fnd += 1
            if fnd > 10:
                return
        

In [10]:
np.seterr(all='raise')
BF()

except


array([[ -5.,  14., -17., -18.,   4.,  19.],
       [ -1.,   0.,  16.,  -9.,   3., -14.],
       [ 12.,   0., -16.,  -4.,  -6.,  18.],
       [ -9.,  -4., -17.,  12.,  14.,   0.],
       [  0.,   2.,   0.,   0., -15.,   0.],
       [ 13.,  -5.,  -8.,  -7.,  13., -16.],
       [ -5.,   0.,   3.,   7.,  -7.,   0.],
       [ 19.,  -1., -10.,  18.,  17.,  -9.],
       [-15.,   6.,  -5., -12.,   2.,   4.],
       [-15., -19., -13., -14.,   0.,  -1.]])

In [13]:
mat.astype(int)

array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]])

In [16]:
mat_pull_2 = np.array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]], dtype=float)

In [17]:
_,_,_,_, info = pluq_ids(mat_pull_2, debug=False)
print info

FloatingPointError: divide by zero encountered in double_scalars

In [11]:
def zero(a, b):
    a = np.arrray([a])
    b = np.arrray([b])
    return a[0]/b[0]

In [12]:
zero(2.3, 0.0)

AttributeError: 'module' object has no attribute 'arrray'